In [2]:
import pandas as pd
import numpy as np
import random
from autocorrect import Speller
import itertools
import re
from LLM import LLM
from PromptBuilder import PromptBuilder
from metadata import language_registers, discussion_tones

2025-04-05 23:53:05.044344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 23:53:05.054902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743889985.068338 4007288 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743889985.072563 4007288 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 23:53:05.085978: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [11]:
dd = pd.read_csv('dataset_generated_multi_symptoms_many_combinations')
dd

,Unnamed: 0,Dialogue_Generated,symptom,description,meta,language_style,Tone,Detail_level,Enumeration,Explicit_symptom,Spelling_errors
0,0,"""I haff veery sevre cracking at the corners o...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Very severe,Neutral/Standard Register,Neutral,2,True,True,False
1,1,"""I've been noticing that the corners of my mo...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Prefer not to answer,Poetic/Literary Register,Fearful,4,False,False,True
2,2,"""I have been experiencing cracking at the cor...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = None,Vulgar Register,Friendly,4,False,True,True
3,3,"""I've been noticing that the corners of my mo...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Not applicable,Vulgar Register,Neutral,1,False,False,True
4,4,"""I've been noticing that the corners of my mo...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Moderate,Informal Register,Friendly,4,False,False,False
5,5,"""I've been noticing that the corners of my mo...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Mild,Neutral/Standard Register,Fearful,5,False,False,False
6,6,"""I've had these little cracks at the corners ...",Cracking at the corners of the mouth (cheilosi...,Presence/Absence,Severity = Severe,Poetic/Literary Register,Fearful,1,False,False,False
7,7,"""I cant swallow properly, its very difficult.""",Difficulty Swallowing,Presence/Absence,Severity = Very severe,Vulgar Register,Confused,1,True,False,True
8,8,"""I have difficulty swallowing.""",Difficulty Swallowing,Presence/Absence,Severity = Prefer not to answer,Poetic/Literary Register,Fearful,4,True,False,True
9,9,"""I have difficulty swallowing food and liquid...",Difficulty Swallowing,Presence/Absence,Severity = None,Informal Register,Fearful,2,False,True,False


In [12]:
a = pd.read_excel('PRO-CTCAE_Questionnaire_Terminology.xls', sheet_name = 'PRO')
index = (a [a['PRO-CTCAE PT'] == 'Pain and swelling at injection site']).index.values[0] +1 
symptoms = a.iloc[:index]['PRO-CTCAE PT'].values

Dict = {}

for symptom in symptoms : 

    try:
        Dict[symptom] = {}
        descriptions_code = a[a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute Code'].values[0].split(" || ")
        descriptions = a [a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute PT'].values[0].split(" || ")
        for description in descriptions :
            Dict[symptom][description]  = a[a['PRO-CTCAE PT'] == description  ]['PRO-CTCAE Value PT'].values[0].split(" || ")
    
    except Exception as e :
        print(e)

'float' object has no attribute 'split'


In [13]:
for el in Dict.keys():
    for el2 in Dict[el].keys():
        if "Not sexually active" in Dict[el][el2]:
            Dict[el][el2].remove("Not sexually active")

del Dict['Other Symptoms']

print("The dictionary has been cleaned and is ready to be used")

The dictionary has been cleaned and is ready to be used


In [14]:
Dict

{'Cracking at the corners of the mouth (cheilosis/cheilitis)': {'Severity': ['Very severe',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Difficulty Swallowing': {'Severity': ['Very severe',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Dry Mouth': {'Severity': ['Very severe',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Hoarseness': {'Severity': ['Very severe',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Mouth/throat sores': {'Interference (with daily activities)': ['Not at all',
   'A little bit',
   'Somewhat',
   'Quite a bit',
   'Very much'],
  'Severity': ['Very severe',
   'Prefer not to answer',
   'None',
   'Not applicable',
   'Moderate',
   'Mild',
   'Severe']},
 'Voice quality changes': {'Presence/Absence': ['Prefer not to answer',
   'Not applicab

In [7]:
model = LLM()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded on device(s): {'': 0}


## Generate one symptom per sentence 

In [ ]:
import itertools

prompt_builder = PromptBuilder()

data = []
c = 0
for symptom, descriptions in Dict.items():

    # Generate all (description, meta) combinations
    description_meta_combinations = list(itertools.product(descriptions.keys(), *descriptions.values()))

    for description, *meta_combinations in description_meta_combinations:

        for meta_set in meta_combinations:

            if c % 100 == 0:
                print(f"Generating {c+100} sentences...")
        
            detail_level = np.random.choice([1, 2, 3, 4, 5])
                        
            enumeration = np.random.choice([True, False], p=[0.2, 0.8])

            explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])


            language_style = random.choice(language_registers)['name']
            tone = random.choice(discussion_tones)['name']
            spelling_errors = random.choice([True, False])

            prompt = prompt_builder.build_prompt(
                symptoms=[symptom],
                description=description,
                meta=meta_set,
                detail_level=detail_level,
                enumeration=enumeration,
                explicit_symptom=explicit_symptom,
                language_style=language_style,
                spelling_errors=spelling_errors,
                tone=tone
            )
            
            phrase_generated = model.generate_text(messages=prompt)

            data.append([
                phrase_generated, symptom, description, meta_set, language_style, 
                tone, detail_level, enumeration, explicit_symptom, spelling_errors
            ])
            c += 1

            df = pd.DataFrame(data, columns=[
                "Dialogue_Generated", "Symptom", "Description", "Meta", 
                "Language_Style", "Tone", "Detail_Level", "Enumeration", 
                "Explicit_Symptom", "Spelling_Errors"
            ])

            df.to_csv("Final_dataset_generated_one_symptom.csv")

Generating 100 sentences...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Generating 200 sentences...


In [8]:
prompt_builder = PromptBuilder()

data = []
c = 0

symptom_list = list(Dict.keys())

# Define how many symptoms per sentence (e.g., 2 to 4 symptoms)
min_symptoms = 2
max_symptoms = 4

for _ in range(100):  # replace this with your desired total number of examples

    # Randomly sample symptoms
    selected_symptoms = random.sample(symptom_list, k=random.randint(min_symptoms, max_symptoms))

    descriptions = []
    metas = []

    # Get description + meta info for each selected symptom
    for symptom in selected_symptoms:

        descriptions_dict = Dict[symptom]

        description = random.choice(list(descriptions_dict.keys()))

        meta_options = descriptions_dict[description]

        meta = random.choice(meta_options)
        
        descriptions.append((symptom, description))
        metas.append(meta)

    if c % 100 == 0:
        print(f"Generated {c+100} sentences...")

    # Parameters shared across the sentence

    detail_level = np.random.choice([1, 2, 3, 4, 5])
    enumeration = np.random.choice([True, False], p=[0.2, 0.8])
    explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])
    language_style = random.choice(language_registers)['name']
    tone = random.choice(discussion_tones)['name']
    spelling_errors = random.choice([True, False])

    # Build the prompt using all symptoms
    symptoms_only = [s[0] for s in descriptions]
    descriptions_only = [s[1] for s in descriptions]
    
    prompt = prompt_builder.build_prompt(
        symptoms=symptoms_only,
        description=descriptions_only,  # assumes you support a list of descriptions
        meta=metas,
        detail_level=detail_level,
        enumeration=enumeration,
        explicit_symptom=explicit_symptom,
        language_style=language_style,
        spelling_errors=spelling_errors,
        tone=tone
    )

    phrase_generated = model.generate_text(messages=prompt)

    data.append([
        phrase_generated, symptoms_only, descriptions_only, metas, 
        language_style, tone, detail_level, enumeration, 
        explicit_symptom, spelling_errors
    ])
    c += 1

df = pd.DataFrame(data, columns=[
    "Dialogue_Generated", "Symptoms", "Descriptions", "Metas", 
    "Language_Style", "Tone", "Detail_Level", "Enumeration", 
    "Explicit_Symptom", "Spelling_Errors"
])

Generated 100 sentences...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
df.to_csv('New_generated_multi_symptoms_dataset.csv')

## Many combinations of description and metas :

In [8]:
def get_combinations(symptom, symptom_data):
    # Extract the attribute lists for the symptom
    attributes = symptom_data[symptom]
    # Get the attribute names (keys) and corresponding lists (values)
    attribute_names = list(attributes.keys())
    attribute_values = list(attributes.values())
    
    # Generate all combinations using itertools.product
    combinations = list(itertools.product(*attribute_values))
    
    # Optionally, pair each combination with its corresponding attribute names:
    combinations_named = [
        dict(zip(attribute_names, combination)) for combination in combinations
    ]
    
    return combinations_named

In [18]:

data = []

for symptom in Dict.keys():

    combinaisons = get_combinations(symptom, Dict)

    for combi in combinaisons :

        descriptions =  list(combi.keys())

        meta_descriptions = list(combi.items())

        meta_str = " , ".join([f"{k} = {v}" for k, v in meta_descriptions])
        
        detail_level = np.random.choice([1,2,3,4,5])

        enumeration = np.random.choice([True, False], p=[0.2, 0.8])

        explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])

        language_style = random.choice(language_registers)['name']

        tone = random.choice(discussion_tones)['name']

        spelling_errors = np.random.choice([True, False], p=[0.7, 0.3])

        prompt = PromptBuilder().build_prompt([symptom], meta_str, detail_level, enumeration, explicit_symptom, language_style, spelling_errors,
         tone) 
        
        phrase_generated = model.generate_text(messages = prompt)

        data.append([phrase_generated , symptom, descriptions, meta_str, language_style, tone, detail_level, enumeration, explicit_symptom, spelling_errors])

        df = pd.DataFrame(data, columns=["Dialogue_Generated", "symptom", "description", "meta", "language_style", "Tone", "Detail_level", "Enumeration", "Explicit_symptom", "Spelling_errors"])
        df.to_csv("dataset_generated_multi_symptoms_many_combinations")

KeyboardInterrupt: 

In [ ]:
prompt_builder = PromptBuilder()

data = []
mean = 3
std = 1
c = 0

symptom_list = list(Dict.keys())

# Define how many symptoms per sentence (e.g., 2 to 4 symptoms)
min_symptoms = 2
max_symptoms = 5

for _ in range(100):  # replace this with your desired total number of examples

    # Randomly sample symptoms
    num_symptoms = int(np.clip(np.random.normal(loc=mean, scale=std), min_symptoms, max_symptoms))

    selected_symptoms = random.choices(symptom_list,k=num_symptoms)

    # selected_symptoms = random.sample(symptom_list, k=random.randint(min_symptoms, max_symptoms))

    descriptions = []
    metas = []

    # Get description + meta info for each selected symptom
    for symptom in selected_symptoms:

        descriptions_dict = Dict[symptom]

        description = random.choice(list(descriptions_dict.keys()))

        meta_options = descriptions_dict[description]

        meta = random.choice(meta_options)
        
        descriptions.append((symptom, description))
        metas.append(meta)

    if c % 100 == 0:
        print(f"Generated {c+100} sentences...")

    # Parameters shared across the sentence

    detail_level = np.random.choice([1, 2, 3, 4, 5])
    enumeration = np.random.choice([True, False], p=[0.2, 0.8])
    explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])
    language_style = random.choice(language_registers)['name']
    tone = random.choice(discussion_tones)['name']
    spelling_errors = random.choice([True, False])

    # Build the prompt using all symptoms
    symptoms_only = [s[0] for s in descriptions]
    descriptions_only = [s[1] for s in descriptions]
    
    prompt = prompt_builder.build_prompt(
        symptoms=symptoms_only,
        description=descriptions_only,  # assumes you support a list of descriptions
        meta=metas,
        detail_level=detail_level,
        enumeration=enumeration,
        explicit_symptom=explicit_symptom,
        language_style=language_style,
        spelling_errors=spelling_errors,
        tone=tone
    )

    phrase_generated = model.generate_text(messages=prompt)

    data.append([
        phrase_generated, symptoms_only, descriptions_only, metas, 
        language_style, tone, detail_level, enumeration, 
        explicit_symptom, spelling_errors
    ])
    c += 1

df = pd.DataFrame(data, columns=[
    "Dialogue_Generated", "Symptoms", "Descriptions", "Metas", 
    "Language_Style", "Tone", "Detail_Level", "Enumeration", 
    "Explicit_Symptom", "Spelling_Errors"
])

Generated 100 sentences...


KeyboardInterrupt: 

In [25]:
df = pd.DataFrame(data, columns=[
    "Dialogue_Generated", "Symptoms", "Descriptions", "Metas", 
    "Language_Style", "Tone", "Detail_Level", "Enumeration", 
    "Explicit_Symptom", "Spelling_Errors"
])
df

,Dialogue_Generated,Symptoms,Descriptions,Metas,Language_Style,Tone,Detail_Level,Enumeration,Explicit_Symptom,Spelling_Errors
0,"""I cant sleep no more, I cant breathe proper,...","[Insomnia, Shortness of breath, Change in usua...","[Interference (with daily activities), Severit...","[Not at all, Mild, Yes, Prefer not to answer]",Vulgar Register,Fearful,1,True,False,True
1,"""I feel so numb and sad, it's like my heart i...","[Sad, Numbness & tingling]","[Severity, Severity]","[Moderate, Not applicable]",Poetic/Literary Register,Neutral,2,False,True,True
2,"""My skin itches all the time, especially at n...","[Itching, Ejaculation, Body odor]","[Severity, Frequency, Severity]","[Severe, Occasionally, Prefer not to answer]",Informal Register,Angry,5,True,False,False
3,"""My body is aching all over, and I feel like ...","[Muscle pain, Vomiting, Missed expected menstr...","[Severity, Severity, Presence/Absence, Severity]","[Severe, Not applicable, Yes, Mild]",Poetic/Literary Register,Confused,1,False,True,False
4,"""I have been experiencing acne on my face for...","[Acne, Nail ridging, General pain]","[Severity, Presence/Absence, Severity]","[None, Yes, None]",Formal Register,Confused,3,False,False,False
5,"""I've been feeling really sick lately, I've b...","[Vomiting, Heart palpitations]","[Frequency, Frequency]","[Prefer not to answer, Rarely]",Informal Register,Friendly,2,False,False,False
6,"""I've been losing hair quite a bit lately, an...","[Hair loss, Urinary incontinence]","[Amount, Interference (with daily activities)]","[Very much, Quite a bit]",Neutral/Standard Register,Neutral,3,False,False,True
7,"""I've been having this weird discharge from m...","[Vaginal discharge, Diarrhea]","[Amount, Frequency]","[Not at all, Prefer not to answer]",Vulgar Register,Friendly,5,False,False,False
8,"""I've got this red, sore, swollen lump on my ...","[Pain and swelling at injection site, Radiatio...","[Presence/Absence, Severity]","[Not applicable, Very severe]",Vulgar Register,Neutral,5,False,False,False
9,"""My nails have turned a sickly shade of blue,...","[Nail discoloration, Visual floaters, Abdomina...","[Presence/Absence, Presence/Absence, Frequency]","[No, Prefer not to answer, Frequently]",Poetic/Literary Register,Confused,1,False,False,False


In [32]:
df['Dialogue_Generated'].iloc[23]

' "I\'ve been having trouble sleeping lately. I usually fall asleep within 20 minutes of going to bed, but I wake up after 2 hours and can\'t fall back asleep. I\'m also noticing a white discharge from my vagina, which I\'ve never had before. It\'s not itchy or painful, but I\'m worried that it might be a sign of an infection."'